In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain_core.embeddings import Embeddings

import os
import pickle
import os.path
import requests

In [4]:
from typing_extensions import TypedDict
from typing import Any, Annotated, Literal, List, Dict, TypeVar
from pydantic import BaseModel, Field, field_validator
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from IPython.display import Image, display
import time
import json
from datetime import datetime, timedelta
import operator
from enum import Enum
import re

from langchain_core.tools import tool
from langchain_core.runnables.graph import MermaidDrawMethod
from langchain_core.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, ToolMessage
from langchain_core.runnables import RunnableLambda, RunnableWithFallbacks


from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.graph.message import AnyMessage, add_messages
from langchain.agents import AgentExecutor, create_tool_calling_agent

In [5]:
api_key = """"""

In [6]:
def new_chat_openai_model(
    model=None,
    base_url=None,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
):
    return ChatOpenAI(
        model="gpt-4o-mini",
        temperature=temperature,
        max_tokens=max_tokens,
        timeout=timeout,
        max_retries=max_retries,
        api_key=api_key,
        rate_limiter=InMemoryRateLimiter(
            requests_per_second=1.0,
            check_every_n_seconds=0.1,
            max_bucket_size=1,
        )
    )

In [7]:
template = PromptTemplate(
    template="Ответь на вопрос: {question}",
    input_variables=["question"],
)

In [8]:
chain = template | new_chat_openai_model()

In [9]:
chain.invoke({'question': 'Сколько будет 2 + 2?'})

AIMessage(content='2 + 2 будет 4.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 21, 'total_tokens': 30, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac', 'finish_reason': 'stop', 'logprobs': None}, id='run-dfdf60e2-13df-497a-a3f2-4ce377da26ea-0', usage_metadata={'input_tokens': 21, 'output_tokens': 9, 'total_tokens': 30, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [10]:
PROMPT = """
Ты аналитик по принятию решений в торговле акциями. 

Твоя задача разделить новости на отдельные части и оставить только те новости по компаниями, которые относятся представленному списку.
<instructions> 
- Раделить на отдельные части новости.
- Оставить только те части новости по компания, которые относятся к списку <allowed_assets>.
- Каждая отдельная часть должна состоять из названия инструмента ISIN-а  и описания.
- В поле rating подставляй занчение от 0 до 10, 0 - сильно негативная новость, 10 - сильно позитивная новость.
- Выйдай суммаризацию по отдельной части, выдай основную мысль поле description.
- Нет пересечения по части текста с <allowed_assets> верни separate_parts: [].
</instructions> 

{format_instructions}

<allowed_assets>
{allowed_assets}
</allowed_assets>

<text>
{text}
</text>
"""

In [11]:
class SperateParts(BaseModel):
    ISIN: str
    description: str
    rating: int

class SperateParts(BaseModel):
    separate_parts: List[SperateParts]

In [12]:
import pandas as pd

In [13]:
df = pd.read_csv(r'C:\llm\examples.csv')

In [14]:
allowed_assets = df.to_csv(sep=';').replace('\"', '')

In [15]:
parser = PydanticOutputParser(pydantic_object=SperateParts)

prompt_template = PromptTemplate(
    template=PROMPT,
    input_variables=["text"],
    partial_variables={
        "format_instructions": parser.get_format_instructions(),
        "allowed_assets": allowed_assets
    },
)

chain = prompt_template | new_chat_openai_model() | parser 

In [16]:
import zipfile

In [17]:
def load_archive(path):
    with zipfile.ZipFile(path, mode="r", compression=zipfile.ZIP_DEFLATED, compresslevel=9) as zip_file: 
        items_file  = zip_file.open('data.json', 'r')
        #text = items_file.read().decode('utf-8')
        #print(items_file)
        return json.load(items_file)

In [18]:
load_archive(r'C:\news\1011280703\1959.zip')

{'id': 1959,
 'text': 'Не берите в магазин пластиковые карты ❌\n\nПодключите Alfa Pay и платите за покупки смартфоном — как раньше. За 1 секунду ❤ \n\nКак платить без карты, если у вас андроид: \n\n1. Настройте Alfa Pay в приложении: Главный экран >> Все мои продукты >> Карта Мир >> Оплата смартфоном. \n2. На кассе поднесите смартфон к терминалу. Вуаля 💫\n\nЕсли у вас айфон, платить без карты тоже можно — просто наклейте наш платёжный стикер с котиком.\n\nРедактор канала: всегда хранит ценные вещи дома 💳\n\n@alfabank',
 'channel_id': 1011280703,
 'date': '2024-01-12T06:01:56+00:00',
 'edit_date': '2024-01-15T07:14:19+00:00',
 'comments': 0,
 'views': 2081668,
 'forwards': 1331,
 'reactions': [{'emoticon': '👍', 'count': 1289},
  {'emoticon': '❤', 'count': 1015},
  {'emoticon': '👎', 'count': 802},
  {'emoticon': '🔥', 'count': 125},
  {'emoticon': '🤔', 'count': 90},
  {'emoticon': '😢', 'count': 51},
  {'emoticon': '🤯', 'count': 30}]}

In [19]:
text = """
Отмена санкций: как это повлияет на рынок и инвесторов

Фондовый рынок активно анализирует заявления лидеров США и России, пытаясь предсказать улучшения в 2025 году. В новом обзоре (https://l.tbank.ru/ustentige) мы разобрали наиболее вероятные сценарии.

✔️ Цены акций уже частично учитывают снижение геополитической премии, но облигации остаются под давлением общего рыночного движения, связанного с уменьшением рисков.

✔️ Укрепление рубля маловероятно даже при смягчении санкций, так как это может привести к росту импорта и спроса на валюту. Ситуация может напомнить 2022 год, когда санкции не всегда негативны для рубля, а их снятие — не гарантия его укрепления.

Возможные сценарии

Снятие санкций против транспорта нефти/СПГ и отмена потолка цен
Вероятность 50%

Макроэкономика: снижение мировых цен на нефть, уменьшение фрахта, умеренный позитив для рубля.

Эмитенты: НОВАТЭК должен выиграть от решения проблем с транспортом, так как они тормозят развитие его СПГ-проектов, на которые компания делает ставку. Нефтяники могут выиграть от снижения расходов на транспорт. Однако рубль может укрепиться, что приведет к убыткам.

Восстановление прямого авиасообщения с западными странами
Вероятность 50%

Макроэкономика: снижение цен на авиабилеты, но негатив для рубля из-за роста расходов на туризм и импорт.

Эмитенты: Аэрофлот может повысить прибыльность благодаря восстановлению международных рейсов и возврату «пролетных» платежей.

Разморозка заблокированных активов и восстановление движения капитала
Вероятность 20%

Макроэкономика: восстановление подключения к SWIFT и нормализация торговых расчетов.

Эмитенты: СПБ Биржа может выиграть от восстановления возможности торговать акциями иностранных компаний. Мосбиржа выиграет от роста ликвидности и возвращения иностранных институционалов. Уменьшение геополитической премии и приток капитала приведут к переоценке мультипликаторов и росту наиболее ликвидных голубых фишек: Сбербанка, Лукойла, Газпрома, Т-Технологий, НОВАТЭКа, Норникеля, Полюса, Роснефти, Яндекса, X5.
"""

In [21]:
chain.invoke({"text": text})

SperateParts(separate_parts=[SperateParts(ISIN='RU0009062285', description='Аэрофлот может повысить прибыльность благодаря восстановлению международных рейсов и возврату «пролетных» платежей.', rating=7), SperateParts(ISIN='RU000A0DKVS5', description='НОВАТЭК должен выиграть от решения проблем с транспортом, так как они тормозят развитие его СПГ-проектов, на которые компания делает ставку.', rating=6), SperateParts(ISIN='RU0009029540', description='Сбербанк может выиграть от роста ликвидности и возвращения иностранных институционалов.', rating=8), SperateParts(ISIN='RU0009024277', description='ЛУКОЙЛ может выиграть от уменьшения геополитической премии и притока капитала.', rating=8), SperateParts(ISIN='RU0007661625', description='Газпром может выиграть от уменьшения геополитической премии и притока капитала.', rating=8), SperateParts(ISIN='RU0007288411', description='Норникель может выиграть от уменьшения геополитической премии и притока капитала.', rating=8), SperateParts(ISIN='RU000A

In [22]:
from tqdm import tqdm

In [23]:
results = []

In [24]:
import os
rootdir = r'C:\news1'
i = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        print(file)
        path = os.path.join(subdir, file)
        data = load_archive(path)
        text = data['text']
        result = None
        try:
            result = chain.invoke({"text": text})
        except:
            pass
        
        if not result is None:
            for part in result.separate_parts:
                results.append({'ISIN': part.ISIN, 'description': part.description, 'rating': part.rating, 'date': data['date'], 'views': data['views']})
                i += 1
                print(i)

8300.zip
8301.zip
1
8302.zip
8303.zip
8308.zip
2
8309.zip
8310.zip
8313.zip
8314.zip
8315.zip
3
4
5
8316.zip
8317.zip
8322.zip
8323.zip
8331.zip
8332.zip
6
8333.zip
8334.zip
7
8335.zip
8336.zip
8337.zip
8
8338.zip
9
10
8339.zip
8344.zip
11
12
13
14
8345.zip
15
16
17
18
8346.zip
8347.zip
8348.zip
8350.zip
8351.zip
8352.zip
19
8357.zip
20
8358.zip
8359.zip
8360.zip
8361.zip
8362.zip
8363.zip
8368.zip
21
8369.zip
8370.zip
22
8371.zip
8372.zip
8373.zip
23
8374.zip
8380.zip
8382.zip
24
25
26
27
8383.zip
8384.zip
8389.zip
8390.zip
28
8391.zip
8392.zip
8393.zip
8394.zip
8395.zip
29
8396.zip
8397.zip
8402.zip
8403.zip
8404.zip
8410.zip
8411.zip
30
31
8412.zip
8413.zip
8414.zip
8415.zip
8420.zip
8421.zip
8423.zip
8424.zip
8425.zip
8427.zip
8428.zip
8429.zip
8430.zip
8431.zip
32
8432.zip
8433.zip
8434.zip
8435.zip
8440.zip
8441.zip
8442.zip
33
34
35
36
8443.zip
8444.zip
8449.zip
8450.zip
8451.zip
8452.zip
8457.zip
8458.zip
8459.zip
8460.zip
8461.zip
8466.zip
8467.zip
8468.zip
8469.zip
8470.zip
8

9569.zip
378
379
9570.zip
9571.zip
9572.zip
380
9573.zip
9574.zip
9575.zip
381
9579.zip
9584.zip
382
383
9585.zip
9586.zip
9587.zip
384
9588.zip
9591.zip
9592.zip
9593.zip
9594.zip
9595.zip
9596.zip
9601.zip
9602.zip


In [27]:
dfk = pd.DataFrame(results)

In [29]:
dfk.to_csv(r'C:\mipt\example.csv', sep=';')